In [1]:
import pandas as pd
import numpy as np

In [2]:
summer_blocks22 = pd.read_excel('../data/summer2022blocks.xlsx', sheet_name='Original')
summer_blocks22_regroup = pd.read_excel('../data/summer2022blocks.xlsx', sheet_name='Optimized')
winter_blocks22_optimized = pd.read_excel('../data/summer2022blocks.xlsx', sheet_name='Winter Optimized')
winter_blocks22 = pd.read_excel('../data/summer2022blocks.xlsx', sheet_name='Winter Original')
summer_blocks22.head()

,Summer 2022 Route,Round Trip Distance km,Trips/day,Total kms/day
0,1-1,11.78,17,200.26
1,1-2,11.78,14,164.92
2,1-3,11.78,6,70.68
3,1C-1,15.50,3,46.50
4,1C-2,15.50,5,77.50


In [3]:
# SCT 2030 Bus Ranges Using 95% design
range_thresholds ={
    'FCEB_TODAY': {'min': 278, 'max': 406},
    'FCEB_2025': {'min': 401, 'max': 586},
    'FCEB_2030': {'min': 614, 'max': 898},
    'FCEB_DD_2030': {'min': 463, 'max': 778},
    'BEB_DDB_450': {'min': 146, 'max': 343},
    'BEB_S_TODAY': {'min': 127, 'max': 286}, #492 kWh
    'BEB_S_2025': {'min': 145, 'max': 328}, #492 kWh
    'BEB_S_2030': {'min': 181, 'max': 409}, #492 kWh
    'BEB_L_TODAY': {'min': 190, 'max': 430}, #738 kWh
    'BEB_L_2025': {'min': 218, 'max': 493}, #738 kWh
    'BEB_L_2030': {'min': 272, 'max': 614}, #738 kWh
    'BEB_EOL_S_TODAY': {'min': 127, 'max': 286}, #492 kWh
    'BEB_EOL_S_2025': {'min': 145, 'max': 328}, #492 kWh
    'BEB_EOL_S_2030': {'min': 181, 'max': 409}, #492 kWh
    'BEB_EOL_L_TODAY': {'min': 190, 'max': 430}, #738 kWh
    'BEB_EOL_L_2025': {'min': 218, 'max': 493}, #738 kWh
    'BEB_EOL_L_2030': {'min': 272, 'max': 614}, #738 kWh
    'FCEB_TODAY_EOL': {'min': 223, 'max': 325}, #20% Degradation
    'FCEB_2025_EOL': {'min': 321, 'max': 470}, #20% Degradation
    'FCEB_2030_EOL': {'min': 492, 'max': 720}, #20% Degradation
}
range_thresholds_cold ={
    'FCEB_TODAY': {'min': 278, 'max': 406},
    'FCEB_2025': {'min': 401, 'max': 586},
    'FCEB_2030': {'min': 614, 'max': 898},
    'BEB_S_TODAY': {'min': 102, 'max': 286}, #492 kWh
    'BEB_S_2025': {'min': 117, 'max': 328}, #492 kWh
    'BEB_S_2030': {'min': 146, 'max': 409}, #492 kWh
    'BEB_L_TODAY': {'min': 153, 'max': 430}, #738 kWh
    'BEB_L_2025': {'min': 175, 'max': 493}, #738 kWh
    'BEB_L_2030': {'min': 219, 'max': 614}, #738 kWh
    'BEB_EOL_S_TODAY': {'min': 71, 'max': 201}, #492 kWh
    'BEB_EOL_S_2025': {'min': 82, 'max': 230}, #492 kWh
    'BEB_EOL_S_2030': {'min': 102, 'max': 287}, #492 kWh
    'BEB_EOL_L_TODAY': {'min': 107, 'max': 301}, #738 kWh
    'BEB_EOL_L_2025': {'min': 123, 'max': 345}, #738 kWh
    'BEB_EOL_L_2030': {'min': 153, 'max': 430}, #738 kWh
    'FCEB_TODAY_EOL': {'min': 223, 'max': 325}, #20% Degradation
    'FCEB_2025_EOL': {'min': 321, 'max': 470}, #20% Degradation
    'FCEB_2030_EOL': {'min': 492, 'max': 720}, #20% Degradation
}

In [4]:
def bus_type_min(df, bus_range_col, range_dict ):
    c1 = (bus_range_col <= range_dict['BEB_450_MIN'])
    c2 = (bus_range_col <= range_dict['BEB_675_MIN'])
    c3 = (bus_range_col <= range_dict['FCEB_2030_MIN'])
    min_conditions = [c1, c2, c3]

    v1 = '450 kWh BEB'
    v2 = '675 kWh BEB'
    v3 = 'FCEB'
    min_values = [v1, v2, v3]

    df['Min Range Bus Type'] = np.select(min_conditions, min_values, default='No ZEB')

    return df

def bus_type_max(df, bus_range_col, range_dict):

    c4 = (bus_range_col <= range_dict['BEB_450_MAX'])
    c5 = (bus_range_col <= range_dict['BEB_675_MAX'])
    c6 = (bus_range_col <= range_dict['FCEB_2030_MAX'])
    max_conditions = [c4, c5, c6]

    v1 = '450 kWh BEB'
    v2 = '675 kWh BEB'
    v3 = 'FCEB'
    max_values = [v1, v2, v3]

    df['Max Range Bus Type'] = np.select(max_conditions, max_values, default='No ZEB')

    return df

In [5]:
def bus_types(df, df_range_col, bus_type_dict, range_thresholds, row_label=None, condition='min'):
    """
    Function that takes a Dataframe object and returns
    :param df: Dataframe object that contains routes with a column for route length
    :param df_range_col: Dataframe column name of length column
    :param bus_type_dict: Dictionary passed to function. Keys are name of bus routes for the Dataframe, values are names of keys for the ranges you wish to use, passed in the range_thresholds dict
    :param range_thresholds: Dictionary
    :param condition: Default is 'min'. Specify 'min' or 'max' condition for range
    :return: Dataframe with additional column for Bus Type that meets route requirements
    """
    conditions = list()
    for bus_range in bus_type_dict.values():
        range_val = range_thresholds[bus_range][condition]
        conditions.append((df[df_range_col] <= range_val))

    if row_label:
        df[row_label] = np.select(conditions, bus_type_dict.keys(), default='No ZEB')
    elif condition == 'min':
        df['Bus Type Min'] = np.select(conditions, bus_type_dict.keys(), default='No ZEB')
    elif condition == 'max':
        df['Bus Type Max'] = np.select(conditions, bus_type_dict.keys(), default='No ZEB')

    return df

In [6]:
class RouteScenario:

    def __init__(self, bus_ranges, bus_selection, schedule, degradation, year, block_scenario): # df, bus_selection,  row_label, condition='min'):
        """
        :param bus_ranges (dictionary): Contains values for each bus type under min and max temperature conditions
        :param bus_selection: Dictionary passed to function. Keys are name of bus routes for the Dataframe, values are names of keys for the ranges you wish to use, passed in the range_thresholds dict
        :param schedule (string): What time of year is the schedule representative of
        :param degradation (boolean): Degradation present in selected ranges
        :param year (number): Year of Technology used in scenario
        """

        self.bus_ranges = bus_ranges
        self.schedule = schedule
        self.degradation = degradation
        self.year = year
        self.block_scenario = block_scenario
        self.bus_selection = bus_selection
        self.beb_small_range = bus_selection['BEB 492 kWh']
        self.beb_large_range = bus_selection['BEB 738 kWh']
        self.fceb_range = bus_selection['FCEB']
        self.composition = ''


    def bus_types(self, df, row_label, condition):
        """
        Function that takes a Dataframe object and returns
        :param df: Dataframe object that contains routes with a column for route length
        :param row_label: Dataframe column name of length column
        :param condition: Default is 'min'. Specify 'min' or 'max' condition for range
        :return: Dataframe with additional column for Bus Type that meets route requirements
        """

        conditions = list()
        bus_ranges = self.bus_ranges
        bus_selection = self.bus_selection

        for bus_range in bus_selection.values():
            range_val = bus_ranges[bus_range][condition]
            conditions.append((df[row_label] <= int(range_val)))


        df['Composition'] = np.select(conditions, bus_selection.keys(), default='No ZEB')
        self.composition =df['Composition']

        return self.composition


In [7]:
scenarios = [{
    #
    # 2022 Summer Scenarios
    #

    'bus_selection': {'BEB 492 kWh': 'BEB_S_TODAY', 'BEB 738 kWh': 'BEB_L_TODAY', 'FCEB': 'FCEB_TODAY'},
    'schedule': 'Summer',
    'year': 2022,
    'degradation': False,
    'block_scenario': 'Existing'
},{
    'bus_selection': {'BEB 492 kWh': 'BEB_EOL_S_TODAY', 'BEB 738 kWh': 'BEB_EOL_L_TODAY', 'FCEB': 'FCEB_TODAY_EOL'},
    'schedule': 'Summer',
    'year': 2022,
    'degradation': True,
    'block_scenario': 'Existing'
},{
    'bus_selection': {'BEB 492 kWh': 'BEB_S_TODAY', 'BEB 738 kWh': 'BEB_L_TODAY', 'FCEB': 'FCEB_TODAY'},
    'schedule': 'Summer',
    'year': 2022,
    'degradation': False,
    'block_scenario': 'Optimized'
},{
    'bus_selection': {'BEB 492 kWh': 'BEB_EOL_S_TODAY', 'BEB 738 kWh': 'BEB_EOL_L_TODAY', 'FCEB': 'FCEB_TODAY_EOL'},
    'schedule': 'Summer',
    'year': 2022,
    'degradation': True,
    'block_scenario': 'Optimized'
},
    #
    # 2025 Summer Scenarios
    #

    {
    'bus_selection': {'BEB 492 kWh': 'BEB_S_2025', 'BEB 738 kWh': 'BEB_L_2025', 'FCEB': 'FCEB_2025'},
    'schedule': 'Summer',
    'year': 2025,
    'degradation': False,
    'block_scenario': 'Existing'
},{
    'bus_selection': {'BEB 492 kWh': 'BEB_EOL_S_2025', 'BEB 738 kWh': 'BEB_EOL_L_2025', 'FCEB': 'FCEB_2025_EOL'},
    'schedule': 'Summer',
    'year': 2025,
    'degradation': True,
    'block_scenario': 'Existing'
},{
    'bus_selection': {'BEB 492 kWh': 'BEB_S_2025', 'BEB 738 kWh': 'BEB_L_2025', 'FCEB': 'FCEB_2025'},
    'schedule': 'Summer',
    'year': 2025,
    'degradation': False,
    'block_scenario': 'Optimized'
},{
    'bus_selection': {'BEB 492 kWh': 'BEB_EOL_S_2025', 'BEB 738 kWh': 'BEB_EOL_L_2025', 'FCEB': 'FCEB_2025_EOL'},
    'schedule': 'Summer',
    'year': 2025,
    'degradation': True,
    'block_scenario': 'Optimized'
},
    #
    # 2030 Summer Blocks
    #

    {
    'bus_selection': {'BEB 492 kWh': 'BEB_S_2030', 'BEB 738 kWh': 'BEB_L_2030', 'FCEB': 'FCEB_2030'},
    'schedule': 'Summer',
    'year': 2030,
    'degradation': False,
    'block_scenario': 'Existing'
},{
    'bus_selection': {'BEB 492 kWh': 'BEB_EOL_S_2030', 'BEB 738 kWh': 'BEB_EOL_L_2030', 'FCEB': 'FCEB_2030_EOL'},
    'schedule': 'Summer',
    'year': 2030,
    'degradation': True,
    'block_scenario': 'Existing'
},{
    'bus_selection': {'BEB 492 kWh': 'BEB_S_2030', 'BEB 738 kWh': 'BEB_L_2030', 'FCEB': 'FCEB_2030'},
    'schedule': 'Summer',
    'year': 2030,
    'degradation': False,
    'block_scenario': 'Optimized'
},{
    'bus_selection': {'BEB 492 kWh': 'BEB_EOL_S_2030', 'BEB 738 kWh': 'BEB_EOL_L_2030', 'FCEB': 'FCEB_2030_EOL'},
    'schedule': 'Summer',
    'year': 2030,
    'degradation': True,
    'block_scenario': 'Optimized'
},{
    #
    # 2022 Winter Scenarios
    #

    'bus_selection': {'BEB 492 kWh': 'BEB_S_TODAY', 'BEB 738 kWh': 'BEB_L_TODAY', 'FCEB': 'FCEB_TODAY'},
    'schedule': 'Winter',
    'year': 2022,
    'degradation': False,
    'block_scenario': 'Existing'
},{
    'bus_selection': {'BEB 492 kWh': 'BEB_EOL_S_TODAY', 'BEB 738 kWh': 'BEB_EOL_L_TODAY', 'FCEB': 'FCEB_TODAY_EOL'},
    'schedule': 'Winter',
    'year': 2022,
    'degradation': True,
    'block_scenario': 'Existing'
},{
    'bus_selection': {'BEB 492 kWh': 'BEB_S_TODAY', 'BEB 738 kWh': 'BEB_L_TODAY', 'FCEB': 'FCEB_TODAY'},
    'schedule': 'Winter',
    'year': 2022,
    'degradation': False,
    'block_scenario': 'Optimized'
},{
    'bus_selection': {'BEB 492 kWh': 'BEB_EOL_S_TODAY', 'BEB 738 kWh': 'BEB_EOL_L_TODAY', 'FCEB': 'FCEB_TODAY_EOL'},
    'schedule': 'Winter',
    'year': 2022,
    'degradation': True,
    'block_scenario': 'Optimized'
},
    #
    # 2025 Winter Scenarios
    #

    {
    'bus_selection': {'BEB 492 kWh': 'BEB_S_2025', 'BEB 738 kWh': 'BEB_L_2025', 'FCEB': 'FCEB_2025'},
    'schedule': 'Winter',
    'year': 2025,
    'degradation': False,
    'block_scenario': 'Existing'
},{
    'bus_selection': {'BEB 492 kWh': 'BEB_EOL_S_2025', 'BEB 738 kWh': 'BEB_EOL_L_2025', 'FCEB': 'FCEB_2025_EOL'},
    'schedule': 'Winter',
    'year': 2025,
    'degradation': True,
    'block_scenario': 'Existing'
},{
    'bus_selection': {'BEB 492 kWh': 'BEB_S_2025', 'BEB 738 kWh': 'BEB_L_2025', 'FCEB': 'FCEB_2025'},
    'schedule': 'Winter',
    'year': 2025,
    'degradation': False,
    'block_scenario': 'Optimized'
},{
    'bus_selection': {'BEB 492 kWh': 'BEB_EOL_S_2025', 'BEB 738 kWh': 'BEB_EOL_L_2025', 'FCEB': 'FCEB_2025_EOL'},
    'schedule': 'Winter',
    'year': 2025,
    'degradation': True,
    'block_scenario': 'Optimized'
},
    #
    # 2030 Winter Blocks
    #

    {
    'bus_selection': {'BEB 492 kWh': 'BEB_S_2030', 'BEB 738 kWh': 'BEB_L_2030', 'FCEB': 'FCEB_2030'},
    'schedule': 'Winter',
    'year': 2030,
    'degradation': False,
    'block_scenario': 'Existing'
},{
    'bus_selection': {'BEB 492 kWh': 'BEB_EOL_S_2030', 'BEB 738 kWh': 'BEB_EOL_L_2030', 'FCEB': 'FCEB_2030_EOL'},
    'schedule': 'Winter',
    'year': 2030,
    'degradation': True,
    'block_scenario': 'Existing'
},{
    'bus_selection': {'BEB 492 kWh': 'BEB_S_2030', 'BEB 738 kWh': 'BEB_L_2030', 'FCEB': 'FCEB_2030'},
    'schedule': 'Winter',
    'year': 2030,
    'degradation': False,
    'block_scenario': 'Optimized'
},{
    'bus_selection': {'BEB 492 kWh': 'BEB_EOL_S_2030', 'BEB 738 kWh': 'BEB_EOL_L_2030', 'FCEB': 'FCEB_2030_EOL'},
    'schedule': 'Winter',
    'year': 2030,
    'degradation': True,
    'block_scenario': 'Optimized'
},
]

In [8]:
scenario_objs = list()
for scenario in scenarios:
    route_scenario = RouteScenario(range_thresholds_cold, scenario['bus_selection'], scenario['schedule'], scenario['degradation'], scenario['year'], scenario['block_scenario'])
    scenario_objs.append(route_scenario)


In [9]:
# scenario = scenarios[0]
# route_scenario = RouteScenario(range_thresholds, scenario['bus_selection'], scenario['schedule'], scenario['degradation'], scenario['year'], scenario['block_scenario'])
# route_scenario.bus_types(summer_blocks22, 'Total kms/day', 'min' )

In [10]:
summer_blocks22.shape

(26, 4)

In [11]:
df = pd.DataFrame()
i = 0
for scenario in scenario_objs:
    df.loc[i, 'Block Scenario'] = scenario.block_scenario
    df.loc[i, 'Degradation'] = scenario.degradation
    df.loc[i, 'Schedule'] = scenario.schedule
    df.loc[i, 'Technology'] = scenario.year

    label = 'Total kms/day'
    if scenario.schedule == 'Summer':
        if scenario.block_scenario == 'Existing':
            blocks = summer_blocks22
            min_composition = scenario.bus_types(blocks, label, 'min' )
            max_composition = scenario.bus_types(blocks, label, 'max' )
            max_length = max(blocks[label])
        elif scenario.block_scenario == 'Optimized':
            blocks = summer_blocks22_regroup
            min_composition = scenario.bus_types(blocks, label, 'min' )
            max_composition = scenario.bus_types(blocks, label, 'max' )
            max_length = max(blocks[label])

    if scenario.schedule == 'Winter':
        if scenario.block_scenario == 'Existing':
            blocks = winter_blocks22
            min_composition = scenario.bus_types(blocks, label, 'min' )
            max_composition = scenario.bus_types(blocks, label, 'max' )
            max_length = max(blocks[label])
        elif scenario.block_scenario == 'Optimized':
            blocks = winter_blocks22_optimized
            min_composition = scenario.bus_types(blocks, label, 'min' )
            max_composition = scenario.bus_types(blocks, label, 'max' )
            max_length = max(blocks[label])

    df.loc[i, 'Number of Blocks'] = len(min_composition)
    df.loc[i, 'Max Block Length'] = max_length
    df.loc[i, 'BEB Small Min'] = scenario.bus_ranges[scenario.bus_selection['BEB 492 kWh']]['min']
    df.loc[i, 'BEB Large Min'] = scenario.bus_ranges[scenario.bus_selection['BEB 738 kWh']]['min']
    df.loc[i, 'FCEB Min'] = scenario.bus_ranges[scenario.bus_selection['FCEB']]['min']
    df.loc[i, 'BEB Small Max'] = scenario.bus_ranges[scenario.bus_selection['BEB 492 kWh']]['max']
    df.loc[i, 'BEB Large Max'] = scenario.bus_ranges[scenario.bus_selection['BEB 738 kWh']]['max']
    df.loc[i, 'FCEB Max'] = scenario.bus_ranges[scenario.bus_selection['FCEB']]['max']
    try:
        df.loc[i, '% BEB Small Min'] = min_composition.value_counts()['BEB 492 kWh'] / df.loc[i, 'Number of Blocks']
    except:
        df.loc[i, '% BEB Small Min'] = 0
    try:
        df.loc[i, '% BEB Large Min'] = min_composition.value_counts()['BEB 738 kWh'] / df.loc[i, 'Number of Blocks']
    except:
        df.loc[i, '% BEB Large Min'] = 0
    try:
        df.loc[i, '% FCEB Min'] = min_composition.value_counts()['FCEB'] / df.loc[i, 'Number of Blocks']
    except:
        df.loc[i, '% FCEB Min'] = 0
    df.loc[i, '% BEB Small Max'] = max_composition.value_counts()['BEB 492 kWh'] / df.loc[i, 'Number of Blocks']
    try:
        df.loc[i, '% BEB Large Max'] = max_composition.value_counts()['BEB 738 kWh'] / df.loc[i, 'Number of Blocks']
    except:
        df.loc[i, '% BEB Large Max'] = 0
    try:
        df.loc[i, '% FCEB Max'] = max_composition.value_counts()['FCEB'] / df.loc[i, 'Number of Blocks']
    except:
         df.loc[i, '% FCEB Max'] = 0
    i += 1
df

,Block Scenario,Degradation,Schedule,Technology,Number of Blocks,Max Block Length,BEB Small Min,BEB Large Min,FCEB Min,BEB Small Max,BEB Large Max,FCEB Max,% BEB Small Min,% BEB Large Min,% FCEB Min,% BEB Small Max,% BEB Large Max,% FCEB Max
0,Existing,False,Summer,2022.0,26.0,776.40,102.0,153.0,278.0,286.0,430.0,406.0,0.230769,0.038462,0.153846,0.423077,0.307692,0.000000
1,Existing,True,Summer,2022.0,26.0,776.40,71.0,107.0,223.0,201.0,301.0,325.0,0.153846,0.076923,0.192308,0.423077,0.038462,0.000000
2,Optimized,False,Summer,2022.0,33.0,379.21,102.0,153.0,278.0,286.0,430.0,406.0,0.181818,0.060606,0.181818,0.424242,0.575758,0.000000
3,Optimized,True,Summer,2022.0,33.0,379.21,71.0,107.0,223.0,201.0,301.0,325.0,0.121212,0.060606,0.181818,0.363636,0.181818,0.121212
4,Existing,False,Summer,2025.0,26.0,776.40,117.0,175.0,401.0,328.0,493.0,586.0,0.269231,0.115385,0.346154,0.461538,0.307692,0.038462
5,Existing,True,Summer,2025.0,26.0,776.40,82.0,123.0,321.0,230.0,345.0,470.0,0.192308,0.076923,0.192308,0.423077,0.192308,0.153846
6,Optimized,False,Summer,2025.0,33.0,379.21,117.0,175.0,401.0,328.0,493.0,586.0,0.212121,0.121212,0.666667,0.666667,0.333333,0.000000
7,Optimized,True,Summer,2025.0,33.0,379.21,82.0,123.0,321.0,230.0,345.0,470.0,0.151515,0.060606,0.454545,0.363636,0.424242,0.212121
8,Existing,False,Summer,2030.0,26.0,776.40,146.0,219.0,614.0,409.0,614.0,898.0,0.269231,0.153846,0.500000,0.730769,0.192308,0.076923
9,Existing,True,Summer,2030.0,26.0,776.40,102.0,153.0,492.0,287.0,430.0,720.0,0.230769,0.038462,0.500000,0.423077,0.307692,0.192308


In [13]:
df.to_excel('../data/fleet_comp/scenario_matrix.xlsx')
df.to_excel(r'C:\Users\tdickens\OneDrive - Hydrogen Technology and Engineering Corporation\Active Customers\ETS\08 - Fleet Transition Plans\Roam\06 Route Analysis\scenario_matrix.xlsx')